# Master Data Science for Business - Data Science Consulting - Session 2 

# Notebook 3: 

# Web Scraping with Scrapy: Getting reviews from TripAdvisor

Now you know how to get data from one review, we want to automate the spider so it will crawl through all pages of reviews, ending with a full spider able to scrape every reviews of the selected parc. You will modify here the parse function since this is where you tell the spider to get the links and to follow them. <br> 

## 1. Importing packages

In [5]:
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.spiders import CrawlSpider, Rule
from scrapy.selector import Selector
import sys
from scrapy.http import Request
from scrapy.linkextractors import LinkExtractor
import json
import logging
import pandas as pd

## 2. Some class and functions

In [6]:
# -*- coding: utf-8 -*-

# Define here the models for your scraped items
#
# See documentation in:
# https://doc.scrapy.org/en/latest/topics/items.html

class HotelreviewsItem(scrapy.Item):
    # define the fields for your item here like:
    rating = scrapy.Field()
    review = scrapy.Field()
    title = scrapy.Field()
    trip_date = scrapy.Field()
    trip_type = scrapy.Field()
    published_date = scrapy.Field()
    hotel_type = scrapy.Field()
    hotel_name = scrapy.Field()
    price_range = scrapy.Field()
    reviewer_id = scrapy.Field()
    review_language = scrapy.Field()

In [7]:
def user_info_splitter(raw_user_info):
    """

    :param raw_user_info:
    :return:
    """

    user_info = {}

    splited_info = raw_user_info.split()
    for element in splited_info:
        converted_element = get_convertible_elements_as_dic(element)
        if converted_element:
            user_info[converted_element[0]] = converted_element[1]

    return user_info

## 2. Creating the JSon pipeline 

In [8]:
#JSon pipeline, you can rename the "trust.jl" to the name of your choice
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('tripadvisor2.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

## 3. Spider


When you go on a TripAdvisor page, you will have 5 reviews per page. Reviews are not fully displayed on the page, so you have to open them (i.e follow the link of the review to tell Scrapy to scrape this page) to scrape them. <br>
This means we will use 2 parsing functions: <br>
-The first one will go on the page of the parc, and get the links of the reviews, as well as the links of all the pages. <br>
-The second one will go on each page of each reviews and scrape them using the parse_item() method. For this one, you will use the function you wrote within Notebook 2.  <br>
<b>To Do</b>: Complete the following code, to scrape all the reviews of one parc. 

In [9]:
class MySpider(CrawlSpider):
    name = 'BasicSpider'
    domain_url = "https://www.tripadvisor.com"
    # allowed_domains = ["https://www.tripadvisor.com"]

    start_urls = [
        "https://www.tripadvisor.fr/Hotel_Review-g5555792-d7107948-Reviews-Center_Parcs_Le_Bois_aux_Daims-Les_Trois_Moutiers_Vienne_Nouvelle_Aquitaine.html"]
    
        #Custom settings to modify settings usually found in the settings.py file 
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'tripadvisor2.json'                        # Used for pipeline 2
    }

    def parse(self, response):

        all_review_pages = response.xpath(
            "//a[contains(@class,'pageNum') and contains(@class,'last')]/@data-offset").extract()

        next_reviews_page_url = #YOU WANT HERE THE LINK OF THE NEXT BUTTON

        next_page_number = eval(#YOU WANT HERE THE NUMBER OF THE NEXT PAGE)
            
        #Since scraping can take some time, we limit here to 10 the number of pages of reviews we want to scrape

        if next_page_number < 10:
            yield scrapy.Request(next_reviews_page_url, callback=self.parse)

        #The following part gets reviews' links on a page
        review_urls = []
        for partial_review_url in response.xpath("XPATH TO GET REVIEW'S URL").extract():
            review_url = response.urljoin(partial_review_url)
            if review_url not in review_urls:
                review_urls.append(review_url)

            yield scrapy.Request(review_url, callback=self.parse_review_page)

    def parse_review_page(self, response):

       ###COPY AND PASTE HERE YOUR PARSE() METHOD FROM NOTEBOOK 2


SyntaxError: invalid syntax (<ipython-input-9-a7baf130caa5>, line 22)

## 4. Crawling

In [6]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(MySpider)
process.start()

2019-01-14 15:40:05 [scrapy.utils.log] INFO: Scrapy 1.5.1 started (bot: scrapybot)
2019-01-14 15:40:05 [scrapy.utils.log] INFO: Versions: lxml 4.3.0.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.19.0, Twisted 18.9.0, Python 3.7.2 (default, Jan  2 2019, 17:07:39) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 18.0.0 (OpenSSL 1.1.1a  20 Nov 2018), cryptography 2.4.2, Platform Windows-10-10.0.16299-SP0
2019-01-14 15:40:05 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'tripadvisor2.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


## 5. Importing and reading data scraped

If you've succeeded, you should see here a dataframe with around 40 entries corresponding to the reviews of the Center Parc you scraped. Congratulations ! 

In [7]:
dfjson = pd.read_json('tripadvisor2.json')
#Previewing DF
dfjson.head()

,hotel_adress,hotel_name,hotel_type,image_url,locid,pid,price_range,published_date,rating,review,review_id,review_language,reviewer_id,title,trip_date,trip_type
0,"{'@type': 'PostalAddress', 'streetAddress': 'R...",Center Parcs Le Bois aux Daims,LodgingBusiness,https://media-cdn.tripadvisor.com/media/photo-...,7107948,38673,92€ - 303€ (Selon les tarifs moyens d'une cha...,13 janvier 2019,4,"Promenade dans les bois, visite des animaux de...",645540060,fr,0A71E262B54FB34066F20C1507EBE1D5,Parfait pour les familles avec enfant en bas-\...,janvier 2019,None
1,"{'@type': 'PostalAddress', 'streetAddress': 'R...",Center Parcs Le Bois aux Daims,LodgingBusiness,https://media-cdn.tripadvisor.com/media/photo-...,7107948,38673,92€ - 303€ (Selon les tarifs moyens d'une cha...,11 janvier 2019,5,Cottage tr\u00e8s confortable et tr\u00e8s bie...,645182320,fr,ED122FCA084E4A62A164DE630959A348,Un s\u00e9jour \u00e0 la fois dynamique et rel...,janvier 2019,None
2,"{'@type': 'PostalAddress', 'streetAddress': 'R...",Center Parcs Le Bois aux Daims,LodgingBusiness,https://media-cdn.tripadvisor.com/media/photo-...,7107948,38673,92€ - 303€ (Selon les tarifs moyens d'une cha...,11 janvier 2019,4,Super pour un s\u00e9jour nature en famille et...,645181005,fr,A65156707EBF9A67330AF3AE3F9659AC,La nature... et les v\u00e9los !!!,décembre 2018,None
3,"{'@type': 'PostalAddress', 'streetAddress': 'R...",Center Parcs Le Bois aux Daims,LodgingBusiness,https://media-cdn.tripadvisor.com/media/photo-...,7107948,38673,92€ - 303€ (Selon les tarifs moyens d'une cha...,12 janvier 2019,3,Nous \u00e9tions 7 personnes dans un cottage t...,645403393,fr,1B452E32F640431AA8602DCAE15E67C3,Correct,décembre 2018,A voyagé en famille
4,"{'@type': 'PostalAddress', 'streetAddress': 'R...",Center Parcs Le Bois aux Daims,LodgingBusiness,https://media-cdn.tripadvisor.com/media/photo-...,7107948,38673,92€ - 303€ (Selon les tarifs moyens d'une cha...,11 janvier 2019,2,L\'arriv\u00e9e dans le cottage n\'a pas \u00e...,645189112,fr,3AC3681D0E609D67780EF9BB905CCBE5,Propret\u00e9 d\u00e9sastreuse,janvier 2019,None


In [8]:
dfjson.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 16 columns):
hotel_adress       45 non-null object
hotel_name         45 non-null object
hotel_type         45 non-null object
image_url          45 non-null object
locid              45 non-null int64
pid                45 non-null int64
price_range        45 non-null object
published_date     45 non-null object
rating             45 non-null int64
review             45 non-null object
review_id          45 non-null int64
review_language    45 non-null object
reviewer_id        45 non-null object
title              45 non-null object
trip_date          45 non-null object
trip_type          17 non-null object
dtypes: int64(4), object(12)
memory usage: 5.7+ KB
